In [110]:
import pandas as pd
# save filepath to variable for easier access
#pump_file_path = '../data/processed/pump_dataset.csv'
# this is a merged full file without missed values
pump_file_path = "../data/processed/pump_dataset_full_cleaned.csv"

# read the data and store data in DataFrame titled pomp_data
pump_data = pd.read_csv(pump_file_path) 
# print a summary of the data in water pomp data
pump_data.describe()

,amount_tsh,gps_height,longitude,latitude,region_code,district_code,population,construction_year,year_recorded,month_recorded,dayofweek_recorded,pump_age,location_cluster
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,35.136692,-5.706033e+00,15.297003,5.629747,179.909983,1996.254495,2011.921667,4.375640,2.939933,15.667172,3.761549
std,2997.574558,693.116350,2.628459,2.946019e+00,17.587406,9.633649,471.482176,10.143783,0.958758,3.029247,1.951627,10.157652,2.820036
min,0.000000,-90.000000,29.219817,-1.164944e+01,1.000000,0.000000,0.000000,1960.000000,2002.000000,1.000000,0.000000,-7.000000,0.000000
25%,0.000000,0.000000,33.249501,-8.540621e+00,5.000000,2.000000,0.000000,1993.000000,2011.000000,2.000000,1.000000,8.000000,1.000000
50%,0.000000,369.000000,35.000424,-5.021597e+00,12.000000,3.000000,25.000000,1996.000000,2012.000000,3.000000,3.000000,15.000000,3.000000
75%,20.000000,1319.250000,37.241238,-3.326156e+00,17.000000,5.000000,215.000000,2004.000000,2013.000000,7.000000,5.000000,19.000000,6.000000
max,350000.000000,2770.000000,40.391390,-2.000000e-08,99.000000,80.000000,30500.000000,2013.000000,2013.000000,12.000000,6.000000,53.000000,9.000000


In [111]:
# see the list of all columns in the dataset to choose the varieables/co.umns
pump_data.columns

Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'wpt_name', 'basin', 'subvillage', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'status_group', 'year_recorded', 'month_recorded', 'dayofweek_recorded',
       'funder_installer', 'pump_age', 'location_cluster'],
      dtype='object')

In [112]:
# Separate target from predictors
y = pump_data.status_group
X = pump_data.drop(['status_group'], axis=1)

# Divide data into training and validation subsets
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_train_full = X_train_full.copy()
X_valid_full = X_valid_full.copy() 

# Drop columns with missing values (simplest approach)
#cols_with_missing = ['id','recorded_by', 'num_private', 'payment']
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [113]:
X_train.head()

,basin,extraction_type_class,management_group,payment_type,water_quality,quality_group,quantity,quantity_group,source_type,source_class,...,latitude,region_code,district_code,population,construction_year,year_recorded,month_recorded,dayofweek_recorded,pump_age,location_cluster
55424,Pangani,gravity,user-group,never pay,soft,good,enough,enough,spring,groundwater,...,-4.783454,4,1,1.0,1994,2011,4,3,17,0
28170,Internal,submersible,user-group,per bucket,salty,salty,insufficient,insufficient,borehole,groundwater,...,-5.309293,21,4,1.0,1997,2013,2,1,16,3
20007,Internal,gravity,user-group,never pay,soft,good,insufficient,insufficient,spring,groundwater,...,-3.217454,2,5,180.0,1974,2013,3,5,39,3
7842,Pangani,gravity,commercial,never pay,soft,good,enough,enough,spring,groundwater,...,-3.141919,3,1,1.0,1988,2013,2,0,25,5
22217,Lake Tanganyika,gravity,user-group,monthly,soft,good,enough,enough,river/lake,surface,...,-4.534989,16,2,320.0,1999,2013,2,5,14,9


In [114]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['basin', 'extraction_type_class', 'management_group', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type', 'waterpoint_type_group']


## Define the quality of each Approch

In [115]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)

In [116]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("Accuracy from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

Accuracy from Approach 1 (Drop categorical variables):


0.7233164983164984


In [117]:
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

MAE from Approach 2 (Ordinal Encoding):
0.8067340067340067


In [118]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

MAE from Approach 3 (One-Hot Encoding):
0.8048821548821549


# Cleaning the Data and test the accuracy

In [119]:
"""
import pandas as pd
# save filepath to variable for easier access
pump_file_path = '../data/processed/pump_dataset.csv'
# read the data and store data in DataFrame titled pomp_data
pump_data_clean = pd.read_csv(pump_file_path) 
# print a summary of the data in water pomp data
pump_data_clean.describe()
"""

"\nimport pandas as pd\n# save filepath to variable for easier access\npump_file_path = '../data/processed/pump_dataset.csv'\n# read the data and store data in DataFrame titled pomp_data\npump_data_clean = pd.read_csv(pump_file_path) \n# print a summary of the data in water pomp data\npump_data_clean.describe()\n"

In [120]:
"""
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.experimental import enable_iterative_imputer  # This enables IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

# Transformer: Replace 0 with random longitude
class LongitudeRandomImputer(BaseEstimator, TransformerMixin):
    def __init__(self, column='longitude', min_val=29.2, max_val=40.4):
        self.column = column
        self.min_val = min_val
        self.max_val = max_val

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.column] = X[self.column].replace(0, np.nan)
        X[self.column] = X[self.column].apply(
            lambda x: np.random.uniform(self.min_val, self.max_val) if pd.isna(x) else x
        )
        return X

# Transformer: Group-based mode imputation with global fallback
class GroupModeImputer(BaseEstimator, TransformerMixin):
    def __init__(self, col, group_cols):
        self.col = col
        self.group_cols = group_cols
        self.global_mode = None

    def fit(self, X, y=None):
        self.global_mode = X[self.col].mode(dropna=True)[0]
        return self

    def transform(self, X):
        X = X.copy()
        try:
            X[self.col] = X.groupby(self.group_cols)[self.col].transform(
                lambda x: x.fillna(x.mode()[0]) if not x.mode().empty else x
            )
        except Exception as e:
            print(f"Warning during group imputation of {self.col}: {e}")
        X[self.col].fillna(self.global_mode, inplace=True)
        return X

# Transformer: Iterative imputer for numeric columns
class IterativeNumericImputer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.imputer = IterativeImputer(estimator=BayesianRidge(), random_state=42, max_iter=10)

    def fit(self, X, y=None):
        self.imputer.fit(X[self.columns])
        return self

    def transform(self, X):
        X = X.copy()
        X[self.columns] = self.imputer.transform(X[self.columns])
        return X
"""


'\nimport numpy as np\nimport pandas as pd\nfrom sklearn.base import BaseEstimator, TransformerMixin\nfrom sklearn.experimental import enable_iterative_imputer  # This enables IterativeImputer\nfrom sklearn.impute import IterativeImputer\nfrom sklearn.linear_model import BayesianRidge\n\n# Transformer: Replace 0 with random longitude\nclass LongitudeRandomImputer(BaseEstimator, TransformerMixin):\n    def __init__(self, column=\'longitude\', min_val=29.2, max_val=40.4):\n        self.column = column\n        self.min_val = min_val\n        self.max_val = max_val\n\n    def fit(self, X, y=None):\n        return self\n\n    def transform(self, X):\n        X = X.copy()\n        X[self.column] = X[self.column].replace(0, np.nan)\n        X[self.column] = X[self.column].apply(\n            lambda x: np.random.uniform(self.min_val, self.max_val) if pd.isna(x) else x\n        )\n        return X\n\n# Transformer: Group-based mode imputation with global fallback\nclass GroupModeImputer(BaseEs

In [121]:
"""
from sklearn.pipeline import Pipeline

# List of all imputers and their logic
imputation_pipeline = Pipeline(steps=[
    ('longitude_random', LongitudeRandomImputer()),

    ('funder_impute', GroupModeImputer('funder', ['region', 'lga', 'ward'])),
    ('installer_impute', GroupModeImputer('installer', ['region', 'funder'])),
    ('subvillage_impute', GroupModeImputer('subvillage', ['lga', 'ward'])),
    ('public_meeting_impute', GroupModeImputer('public_meeting', ['region', 'ward'])),
    ('scheme_name_impute', GroupModeImputer('scheme_name', ['basin', 'region', 'longitude', 'latitude'])),
    ('scheme_management_impute', GroupModeImputer('scheme_management', ['scheme_name', 'source', 'region'])),
    ('permit_impute', GroupModeImputer('permit', ['region', 'source'])),

    ('iterative_numeric', IterativeNumericImputer(['construction_year', 'population', 'latitude', 'longitude', 'gps_height']))
])
"""


"\nfrom sklearn.pipeline import Pipeline\n\n# List of all imputers and their logic\nimputation_pipeline = Pipeline(steps=[\n    ('longitude_random', LongitudeRandomImputer()),\n\n    ('funder_impute', GroupModeImputer('funder', ['region', 'lga', 'ward'])),\n    ('installer_impute', GroupModeImputer('installer', ['region', 'funder'])),\n    ('subvillage_impute', GroupModeImputer('subvillage', ['lga', 'ward'])),\n    ('public_meeting_impute', GroupModeImputer('public_meeting', ['region', 'ward'])),\n    ('scheme_name_impute', GroupModeImputer('scheme_name', ['basin', 'region', 'longitude', 'latitude'])),\n    ('scheme_management_impute', GroupModeImputer('scheme_management', ['scheme_name', 'source', 'region'])),\n    ('permit_impute', GroupModeImputer('permit', ['region', 'source'])),\n\n    ('iterative_numeric', IterativeNumericImputer(['construction_year', 'population', 'latitude', 'longitude', 'gps_height']))\n])\n"

In [122]:
"""
# Separate target from predictors
y = pump_data_clean.status_group
X = pump_data_clean.drop(['status_group'], axis=1)

# Divide data into training and validation subsets
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid= train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
X_train = X_train.copy()
X_valid = X_valid.copy() 
"""


"\n# Separate target from predictors\ny = pump_data_clean.status_group\nX = pump_data_clean.drop(['status_group'], axis=1)\n\n# Divide data into training and validation subsets\nfrom sklearn.model_selection import train_test_split\nX_train, X_valid, y_train, y_valid= train_test_split(X, y, train_size=0.8, test_size=0.2,\n                                                                random_state=0)\nX_train = X_train.copy()\nX_valid = X_valid.copy() \n"

In [123]:
"""
# Fit and transform the training data
X_train_full = imputation_pipeline.fit_transform(X_train_full)

# Only transform the validation set
X_valid_full= imputation_pipeline.transform(X_valid_full)
"""

'\n# Fit and transform the training data\nX_train_full = imputation_pipeline.fit_transform(X_train_full)\n\n# Only transform the validation set\nX_valid_full= imputation_pipeline.transform(X_valid_full)\n'

In [124]:

"""
# Drop columns
cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()"""

'\n# Drop columns\ncols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] \nX_train_full.drop(cols_with_missing, axis=1, inplace=True)\nX_valid_full.drop(cols_with_missing, axis=1, inplace=True)\n\n# "Cardinality" means the number of unique values in a column\n# Select categorical columns with relatively low cardinality (convenient but arbitrary)\nlow_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and \n                        X_train_full[cname].dtype == "object"]\n\n# Select numerical columns\nnumerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in [\'int64\', \'float64\']]\n\n# Keep selected columns only\nmy_cols = low_cardinality_cols + numerical_cols\nX_train = X_train[my_cols].copy()\nX_valid = X_valid[my_cols].copy()'

In [125]:
"""
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)
"""

'\n# Get list of categorical variables\ns = (X_train.dtypes == \'object\')\nobject_cols = list(s[s].index)\n\nprint("Categorical variables:")\nprint(object_cols)\n'

## Define Quality for each approach

In [126]:
"""
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestClassifier(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return accuracy_score(y_valid, preds)
"""

'\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import accuracy_score\n\ndef score_dataset(X_train, X_valid, y_train, y_valid):\n    model = RandomForestClassifier(n_estimators=100, random_state=0)\n    model.fit(X_train, y_train)\n    preds = model.predict(X_valid)\n    return accuracy_score(y_valid, preds)\n'

In [127]:
"""
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("Accuracy from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))
"""

'\ndrop_X_train = X_train.select_dtypes(exclude=[\'object\'])\ndrop_X_valid = X_valid.select_dtypes(exclude=[\'object\'])\n\nprint("Accuracy from Approach 1 (Drop categorical variables):")\nprint(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))\n'

In [128]:
"""
from sklearn.preprocessing import OrdinalEncoder

# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])

print("MAE from Approach 2 (Ordinal Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))
"""

'\nfrom sklearn.preprocessing import OrdinalEncoder\n\n# Make copy to avoid changing original data \nlabel_X_train = X_train.copy()\nlabel_X_valid = X_valid.copy()\n\n# Apply ordinal encoder to each column with categorical data\nordinal_encoder = OrdinalEncoder()\nlabel_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])\nlabel_X_valid[object_cols] = ordinal_encoder.transform(X_valid[object_cols])\n\nprint("MAE from Approach 2 (Ordinal Encoding):") \nprint(score_dataset(label_X_train, label_X_valid, y_train, y_valid))\n'

In [129]:
"""
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

# Ensure all columns have string type
OH_X_train.columns = OH_X_train.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))
"""

'\nfrom sklearn.preprocessing import OneHotEncoder\n\n# Apply one-hot encoder to each column with categorical data\nOH_encoder = OneHotEncoder(handle_unknown=\'ignore\', sparse=False)\nOH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))\nOH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))\n\n# One-hot encoding removed index; put it back\nOH_cols_train.index = X_train.index\nOH_cols_valid.index = X_valid.index\n\n# Remove categorical columns (will replace with one-hot encoding)\nnum_X_train = X_train.drop(object_cols, axis=1)\nnum_X_valid = X_valid.drop(object_cols, axis=1)\n\n# Add one-hot encoded columns to numerical features\nOH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)\nOH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)\n\n# Ensure all columns have string type\nOH_X_train.columns = OH_X_train.columns.astype(str)\nOH_X_valid.columns = OH_X_valid.columns.astype(str)\n\nprint("MAE from Approach 3 (One-Hot Encoding)